# B. Process from the XLSX files

- **3:citations** `input_data/2025-03-28/2025-03-27_dropbox_6_ReproSci/0_last version/2025_Last version_March 19th/3_2025_March 9th_citation_counts.xlsm`
  -  il s’agit des citations : je ne l’ai pas travaillé.  
 - **4:first&last** `input_data/2025-03-28/2025-03-27_dropbox_6_ReproSci/0_last version/2025_Last version_March 19th/4_working first and last.xlsx`
    -   ils’agit d’une liste de 20 auteurs qui sont first and last and mais ne pas prendre en compte ceux qui sont PI.  

In [1]:
import pandas as pd
import pickle
import preprocess_utils
import numpy as np
import json

def clean_authors_claims(df):
    df['Sex'] = df['Sex'].map({1: 'Male', 0: 'Female', '??': np.nan, '?': np.nan})
    df = df.drop(columns=[col for col in df.columns if '%' in col])
    df = df.drop(columns=[col for col in df.columns if 'Unnamed' in col])
    df = df.rename(columns={'Conituinity': 'Continuity', 
                                                "Partially verified":"Partially Verified", 
                                                "*Historical lab after 1998":"Historical lab after 1998"
                                                })
    if "Historical lab" in df.columns:
        df['Historical lab'] = df['Historical lab'].astype('boolean')
    if "Continuity" in df.columns:
        df['Continuity'] = df['Continuity'].astype('boolean')
    if "Become a Pi" in df.columns:
        df.loc[df["Become a Pi"] == "??", "Become a Pi"] = np.nan
        df.loc[df["Become a Pi"] == 0, "Become a Pi"] = False
        df.loc[df["Become a Pi"] == 1, "Become a Pi"] = True
    if "MD" in df.columns:
        df.loc[df["MD"] == "??", "MD"] = np.nan
        df.loc[df["MD"] == "?", "MD"] = np.nan
        df.loc[df["MD"] == 'O', "MD"] = False
        df.loc[df["MD"] == 0, "MD"] = False
        df.loc[df["MD"] == 1, "MD"] = True
    if "Ivy league" in df.columns:
        df.loc[df["Ivy league"] == 0, "Ivy league"] = False
        df.loc[df["Ivy league"] == 1, "Ivy league"] = True

    df = df.dropna(subset=['Name'])
    # drop  Schneider DS+D from names as  Schneider DS and  Schneider D are also there in separate rows
    df = df[~(df['Name'] == 'Schneider DS+D')]
    return df

## B.1. Read the stat author xlsx files
- **0: First** `input_data/2025-03-28/2025-03-27_dropbox_6_ReproSci/0_last version/2025_Last version_March 19th/0_First author cleaned_March 18th.xlsx`
  - Pour les statistiques sur ceux qui deviennent PI, Il ne faut pas prendre en compte ceux qui sont surlignés en bleu car ils sont des premiers auteurs qui sont déjà PI. Normalement ce fichier est bon.  
Avec un peu d’approximation, je trouve que sur 291 first authors: 107 become PI et il y  16  ?? (indéterminés)et 13 qui sont déjà PI en bleu :  ce qui fait 107/262=>40,8% (262=291-16-13)  
- **2: Both** `input_data/2025-03-28/2025-03-27_dropbox_6_ReproSci/0_last version/2025_Last version_March 19th/2_2025_March 9th_stats_author.xlsx`
  - obtenu depuis le site
  -  tu trouveras les last avec de nouveau critères  pour historical (tradition), continuty, first and last.  (pour junior et senior il faut voir par articles dans la database)  
  - Feuille 1 : Parmi les first author (1er feuille) :  
    - Zhenting Zhang et Zhaolin Zhang  sont fusionnés en Zhang Z  
    - Zhipeng Wang et Zhi Wang sont fusionné en Wang Z  
    - Hedengren M et Hedengren M et Hedengren-Olcott M ne sont pas fusionnés
  - Feuille 3: pour les last authors:
    - Il faut fusionner Schneider DS et Schneider D -> c'est les mêmes
    - Historical lab (tradition) or not : il faut pendre le colonne Historical lab after 1998 et comparer 1 (historical, trained in hustorical lab) et 0 (non historical) mais ne pas prendre en compte 2.
    - La colonne E first and Last : compte les chercheurs qui apparaissent comme first author  et last authors. Il faudrait comparer 1 (last author qui ont été first author dans une autre labo mais pas PI)  à  0 (last author withouth being first author)  sans prendre en compte les 2 (first author who were already PI when first author).
  

In [2]:
# read an xlx file in pandas
input_file_folder = "input_data/2025-03-28/2025-03-27_dropbox_6_ReproSci/0_last version/2025_Last version_March 19th"

stat_author_fn = f"{input_file_folder}/2_2025_March 9th_stats_author.xlsx"

first_authors_claims = pd.read_excel(stat_author_fn, sheet_name='First')
leading_authors_claims = pd.read_excel(stat_author_fn, sheet_name='Leading')
leading_authors_claims["Authorship"]= "Leading"
first_authors_claims["Authorship"]= "First"

first_authors_claims = clean_authors_claims(first_authors_claims)
leading_authors_claims = clean_authors_claims(leading_authors_claims)

leading_authors_claims = preprocess_utils.build_author_key(leading_authors_claims, author_name_col="Name", key_col="last_author_key")
first_authors_claims = preprocess_utils.build_author_key(first_authors_claims, author_name_col="Name", key_col="first_author_key")

In [3]:
first_authors_covar_fn = f"{input_file_folder}/0_First author cleaned_March 18th.xlsx"

highlighted_blue = pd.DataFrame(["Gateff E", "Hultmark D", "Imler JL", "Kappler C", "Kurucz E", 
                                "Lanot R", "Meister M", "Nappi AJ", "Reichhart JM", "RIZKI MT", 
                                "Rizki RM", "Schneider DS", "Wegner KM"], columns=["Name"])
highlighted_blue = preprocess_utils.build_author_key(highlighted_blue, author_name_col="Name", key_col="first_author_key")

first_authors_covar = pd.read_excel(first_authors_covar_fn)
first_authors_covar["Name"] = first_authors_covar["AAA_first author"]
first_authors_covar["Become a Pi"] = first_authors_covar["AAA_Become a Pi"]
first_authors_covar = first_authors_covar.drop(columns=["AAA_first author", "AAA_Become a Pi","Source"])

first_authors_covar = clean_authors_claims(first_authors_covar)
print(first_authors_covar["Sex"].unique(), 
        first_authors_covar["MD"].unique(), 
        first_authors_covar["Become a Pi"].unique(), 
        first_authors_covar["Ivy league"].unique())


# Check for duplicates in Name
duplicated_keys = first_authors_covar[first_authors_covar["Name"].duplicated(keep=False)]
# Sort by Name to see duplicates together
duplicated_keys = duplicated_keys.sort_values("Name")
# Print the result
print(f"Found {len(duplicated_keys)} rows with duplicate keys")
print(duplicated_keys[["Name", "Become a Pi", "Sex"]])

# These are put manually int he xlsx file:
first_authors_covar.loc[262, "Name"] = "Wang Zhipeng"
first_authors_covar.loc[263, "Name"] = "Wang Zhi"
first_authors_covar.loc[285, "Name"] = "Zhang Zhenting"
first_authors_covar.loc[286, "Name"] = "Zhang Zhaolin"
first_authors_covar = preprocess_utils.build_author_key(first_authors_covar, author_name_col="Name", key_col="first_author_key")
first_authors_covar

first_authors_covar = preprocess_utils.build_author_key(first_authors_covar, author_name_col="Name", key_col="first_author_key")
first_authors_covar

first_authors_covar["ignore_for_become_a_pi"] = False
for aut_key in highlighted_blue["first_author_key"]:
    first_authors_covar.loc[first_authors_covar["first_author_key"] == aut_key, "ignore_for_become_a_pi"] = True
first_authors_covar

['Male' 'Female' nan] [False nan True] [True False nan] [False True]
Found 4 rows with duplicate keys
        Name Become a Pi     Sex
262   Wang Z       False    Male
263   Wang Z         NaN  Female
285  Zhang Z       False  Female
286  Zhang Z       False  Female


/var/folders/pj/15hxgl0j1wg2w_t5k7bvnq5c0000gn/T/ipykernel_77632/514050009.py:30: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'False' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.loc[df["Ivy league"] == 0, "Ivy league"] = False


,Sex,PhD Post-doc,current job,MD,Affiliation,Country,Ivy league,Name,Become a Pi,first_author_key,ignore_for_become_a_pi
0,Male,Senior Staff,PI,False,Forschungszentrum Borstel,Germany,False,Abdelsadik A,True,abdelsadik a,False
1,Male,Post-doc,PI,False,Harvard Medical School,USA,True,Agaisse H,True,agaisse h,False
2,Female,PhD,Industry,False,University of Massachusetts Medical School,USA,False,Aggarwal K,False,aggarwal k,False
3,NaN,PhD,Senior Staff,False,niversity of Missouri-Kansas City,USA,False,Ao J,NaN,ao j,False
4,Male,Post-doc,PI,False,Harvard Medical School and Massachusetts Gener...,USA,True,Apidianakis Y,True,apidianakis y,False
...,...,...,...,...,...,...,...,...,...,...,...
285,Female,PhD,Academia,False,"Institute of Zoology, Chinese Academy of Sciences",China,False,Zhang Zhenting,False,zhang zhenting,False
286,Female,PhD,Admin,False,University of Kentucky,USA,False,Zhang Zhaolin,False,zhang zhaolin,False
287,Female,PhD,??,False,University of California San Diego,USA,True,Zhao HW,False,zhao hw,False
288,Male,PhD,??,False,University of Houston,USA,False,Zhou Z,NaN,zhou z,False


## B.2 Build First Author tidy-ish dataframe

author_db that are first_author with first_authors_covar

and _then_ claims with that

In [4]:
authors_db = pd.read_csv("preprocessed_data/authors_db.csv")
authors_db.sort_values("author_key")

,author_id,article_id,name,sex,leading_author,first_author,career_stage,expertise_level,author_key
724,745,1535,Abdelsadik A,Male,False,True,PhD,Experienced,abdelsadik a
485,496,2122,Agaisse H,Male,False,True,Post-doc,Experienced,agaisse h
487,499,1882,Agaisse H,Male,True,False,Junior PI,Experienced,agaisse h
15,17,2297,Agaisse H,Male,False,True,Post-doc,Experienced,agaisse h
104,108,1752,Aggarwal K,Female,False,True,PhD,Experienced,aggarwal k
...,...,...,...,...,...,...,...,...,...
754,784,1491,Zhu S,Male,True,False,Junior PI,Newcomer,zhu s
740,714,1756,Zhu S,Male,True,False,Junior PI,Newcomer,zhu s
653,621,1748,Zhu S,Male,True,False,Senior PI,Newcomer,zhu s
588,588,2025,Zhuang ZH,Male,False,True,PhD,Newcomer,zhuang zh


In [5]:
claims_db = pd.read_csv("preprocessed_data/claims_db.csv")
major_claims_df = claims_db[claims_db['assertion_type'] == 'major_claim']
major_claims_df

,id,content,article_id,rank,pmid,authors_txt,title,pmid_article,volume,issue,...,journal_name,impact_factor,first_author_extracted,last_author_extracted,primary_affiliation,assertion_type,label,is_assessed,assessment_type,rank_assessment_type
8,577,#gene:FBgn0000279 is immune inducible and is p...,3103.0,2,NaN,Tryselius Y;Samakovlis C;Kimbrell DA;Hultmark D,"CecC, a cecropin gene expressed during metamor...",1740152.0,204,1,...,European journal of biochemistry,5.4,Tryselius Y,Hultmark D,"Department of Microbiology, Stockholm Universi...",major_claim,Major claim,True,Verified,1.0
30,5845,Hemocyte proliferation and lamellocyte formati...,2204.0,3,NaN,Zettervall CJ;Anderl I;Williams MJ;Palmer R;Ku...,A directed screen for genes involved in Drosop...,15381778.0,101,39,...,Proceedings of the National Academy of Science...,11.1,Zettervall CJ,Hultmark D,"Umeå Centre for Molecular Pathogenesis, Byggna...",major_claim,Major claim,True,Verified,1.0
31,5841,The Ras/MAPK pathway is a main regulator of th...,2204.0,1,NaN,Zettervall CJ;Anderl I;Williams MJ;Palmer R;Ku...,A directed screen for genes involved in Drosop...,15381778.0,101,39,...,Proceedings of the National Academy of Science...,11.1,Zettervall CJ,Hultmark D,"Umeå Centre for Molecular Pathogenesis, Byggna...",major_claim,Major claim,True,Verified,1.0
37,5843,The JAK/STAT and JNK pathways are major regula...,2204.0,2,NaN,Zettervall CJ;Anderl I;Williams MJ;Palmer R;Ku...,A directed screen for genes involved in Drosop...,15381778.0,101,39,...,Proceedings of the National Academy of Science...,11.1,Zettervall CJ,Hultmark D,"Umeå Centre for Molecular Pathogenesis, Byggna...",major_claim,Major claim,True,Verified,1.0
110,545,#gene:FBgn0035976 functions upstream of #gene:...,2520.0,2,NaN,Gottar M;Gobert V;Michel T;Belvin M;Duyk G;Hof...,The Drosophila immune response against Gram-ne...,11912488.0,416,6881,...,Nature,64.8,Gottar M,Royet J,UPR 9022 du Centre National de la Recherche Sc...,major_claim,Major claim,True,Verified,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13261,9082,Ehrlichia chaffeensis establishes a pathogenic...,1593.0,1,NaN,Luce-Fedrow A;Von Ohlen T;Chapes SK,Ehrlichia chaffeensis infections in Drosophila...,19687202.0,77,11,...,Infection and immunity,3.1,Luce-Fedrow A,Chapes SK,"Kansas State University, Division of Biology, ...",major_claim,Major claim,True,"Unchallenged, logically consistent",6.0
13264,9728,B. bassiana inhibits PO activation by preventi...,1532.0,2,NaN,Matskevich AA;Quintin J;Ferrandon D,The Drosophila PRR GNBP3 assembles effector co...,20201042.0,40,5,...,European journal of immunology,5.4,Matskevich AA,Ferrandon D,"Equipe Fondation Recherche Médicale, UPR 9022 ...",major_claim,Major claim,True,"Unchallenged, logically consistent",6.0
13270,1769,"A polyclonal antibody against mammalian IRF-1,...",3004.0,2,NaN,Georgel P;Kappler C;Langley E;Gross I;Nicolas ...,Drosophila immunity. A sequence homologous to ...,7537872.0,23,7,...,Nucleic acids research,14.9,Georgel P,Hoffmann JA,Institut de Biologie Moléculaire et Cellulaire...,major_claim,Major claim,True,"Unchallenged, logically inconsistent",7.0
13273,2906,A pathway in addition to Toll regulates degrad...,2874.0,3,NaN,Wu LP;Anderson KV,Regulated nuclear import of Rel proteins in th...,9510254.0,392,6671,...,Nature,64.8,Wu LP,Anderson KV,"Molecular Biology Program, Memorial Sloan-Kett...",major_claim,Major claim,True,"Unchallenged, logically inconsistent",7.0


In [6]:
len(authors_db[authors_db["first_author"]]["author_key"].unique())

292

In [7]:
first_authors_covar

,Sex,PhD Post-doc,current job,MD,Affiliation,Country,Ivy league,Name,Become a Pi,first_author_key,ignore_for_become_a_pi
0,Male,Senior Staff,PI,False,Forschungszentrum Borstel,Germany,False,Abdelsadik A,True,abdelsadik a,False
1,Male,Post-doc,PI,False,Harvard Medical School,USA,True,Agaisse H,True,agaisse h,False
2,Female,PhD,Industry,False,University of Massachusetts Medical School,USA,False,Aggarwal K,False,aggarwal k,False
3,NaN,PhD,Senior Staff,False,niversity of Missouri-Kansas City,USA,False,Ao J,NaN,ao j,False
4,Male,Post-doc,PI,False,Harvard Medical School and Massachusetts Gener...,USA,True,Apidianakis Y,True,apidianakis y,False
...,...,...,...,...,...,...,...,...,...,...,...
285,Female,PhD,Academia,False,"Institute of Zoology, Chinese Academy of Sciences",China,False,Zhang Zhenting,False,zhang zhenting,False
286,Female,PhD,Admin,False,University of Kentucky,USA,False,Zhang Zhaolin,False,zhang zhaolin,False
287,Female,PhD,??,False,University of California San Diego,USA,True,Zhao HW,False,zhao hw,False
288,Male,PhD,??,False,University of Houston,USA,False,Zhou Z,NaN,zhou z,False


In [8]:
first_authors_db = authors_db[authors_db["first_author"]].copy()
first_author_all = pd.merge(first_authors_db, first_authors_covar, how="left", left_on="author_key", right_on="first_author_key", suffixes=("_db", "_covar"))
print(len(first_authors_db.author_key.unique()), len(first_authors_covar.first_author_key.unique()), len(first_author_all.author_key.unique()))

292 290 292


In [9]:
#first_authors_covar.sort_values("first_author_key").first_author_key.unique()

In [10]:
first_author_all[first_author_all.Name.isna()]

,author_id,article_id,name,sex,leading_author,first_author,career_stage,expertise_level,author_key,Sex,PhD Post-doc,current job,MD,Affiliation,Country,Ivy league,Name,Become a Pi,first_author_key,ignore_for_become_a_pi
30,70,2239,Hedengren-Olcott M,Female,False,True,Post-doc,Experienced,hedengren-olcott m,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
163,338,2311,Pearson AM,Male,False,True,Post-doc,Experienced,pearson am,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
172,365,3447,Rizki TM,Male,True,True,Senior PI,Experienced,rizki tm,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
175,363,3448,Rizki TM,Male,True,True,Senior PI,Experienced,rizki tm,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
178,364,3292,Rizki TM,Male,True,True,Senior PI,Experienced,rizki tm,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
188,362,3273,Rizki TM,Male,True,True,Post-doc,Experienced,rizki tm,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
# merge the claims:
first_author_all = pd.merge(major_claims_df, first_author_all, how="left", left_on="article_id", right_on="article_id", suffixes=("_claim_db", "_aut"))
first_author_all

,id,content,article_id,rank,pmid,authors_txt,title,pmid_article,volume,issue,...,PhD Post-doc,current job,MD,Affiliation,Country,Ivy league,Name,Become a Pi,first_author_key,ignore_for_become_a_pi
0,577,#gene:FBgn0000279 is immune inducible and is p...,3103.0,2,NaN,Tryselius Y;Samakovlis C;Kimbrell DA;Hultmark D,"CecC, a cecropin gene expressed during metamor...",1740152.0,204,1,...,PhD,PI,False,University of Stockholm,Sweden,False,Tryselius Y,False,tryselius y,False
1,5845,Hemocyte proliferation and lamellocyte formati...,2204.0,3,NaN,Zettervall CJ;Anderl I;Williams MJ;Palmer R;Ku...,A directed screen for genes involved in Drosop...,15381778.0,101,39,...,PhD,Industry,False,University of Stockholm,Sweden,False,Zettervall CJ,False,zettervall cj,False
2,5841,The Ras/MAPK pathway is a main regulator of th...,2204.0,1,NaN,Zettervall CJ;Anderl I;Williams MJ;Palmer R;Ku...,A directed screen for genes involved in Drosop...,15381778.0,101,39,...,PhD,Industry,False,University of Stockholm,Sweden,False,Zettervall CJ,False,zettervall cj,False
3,5843,The JAK/STAT and JNK pathways are major regula...,2204.0,2,NaN,Zettervall CJ;Anderl I;Williams MJ;Palmer R;Ku...,A directed screen for genes involved in Drosop...,15381778.0,101,39,...,PhD,Industry,False,University of Stockholm,Sweden,False,Zettervall CJ,False,zettervall cj,False
4,545,#gene:FBgn0035976 functions upstream of #gene:...,2520.0,2,NaN,Gottar M;Gobert V;Michel T;Belvin M;Duyk G;Hof...,The Drosophila immune response against Gram-ne...,11912488.0,416,6881,...,PhD,Industry,False,CNRS University of Marseille,France,False,Gottar M,False,gottar m,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1001,9082,Ehrlichia chaffeensis establishes a pathogenic...,1593.0,1,NaN,Luce-Fedrow A;Von Ohlen T;Chapes SK,Ehrlichia chaffeensis infections in Drosophila...,19687202.0,77,11,...,PhD,??,False,Kansas State University,USA,False,Luce-Fedrow A,False,luce-fedrow a,False
1002,9728,B. bassiana inhibits PO activation by preventi...,1532.0,2,NaN,Matskevich AA;Quintin J;Ferrandon D,The Drosophila PRR GNBP3 assembles effector co...,20201042.0,40,5,...,PhD,??,False,CNRS-University of Strasbourg,France,False,Matskevich AA,False,matskevich aa,False
1003,1769,"A polyclonal antibody against mammalian IRF-1,...",3004.0,2,NaN,Georgel P;Kappler C;Langley E;Gross I;Nicolas ...,Drosophila immunity. A sequence homologous to ...,7537872.0,23,7,...,Post-doc,PI,False,CNRS-University of Strasbourg,France,False,Georgel P,True,georgel p,False
1004,2906,A pathway in addition to Toll regulates degrad...,2874.0,3,NaN,Wu LP;Anderson KV,Regulated nuclear import of Rel proteins in th...,9510254.0,392,6671,...,Post-doc,PI,False,Memorial Sloan-Kettering Cancer Center and the...,USA,True,Wu LP,True,wu lp,False


In [12]:
first_author_all = first_author_all.apply(preprocess_utils.safe_strip)
first_author_all.to_csv("preprocessed_data/first_author_claims.csv", index=False)

# Unused stuff

### Altnernative: Merge author_db.article_id inside the covar xlsx

but not long form...

In [26]:
do_alt = False
if do_alt:
    # create an empty str column in first_authors_covar for article_id
    first_authors_covar["article_id"] = ""
    first_authors_covar["Articles"] = 0

    # TODO: check unmatched first authors from authors_db
    for aut_key in first_authors_covar["first_author_key"].unique():
        assert first_authors_covar[first_authors_covar["first_author_key"] == aut_key].shape[0] == 1, f"💥 {aut_key}, {first_authors_covar[first_authors_covar['first_author_key'] == aut_key].shape[0]}"
        matches = authors_db[authors_db["author_key"] == aut_key]
        fh_matches = matches[matches["first_author"]]
        lh_matches = matches[matches["leading_author"]]
        if len(fh_matches) == 0:
            print(f"💥 {aut_key} not found in authors_covar")
            first_authors_covar.loc[first_authors_covar["first_author_key"] == aut_key, "article_id"] = pd.NA
        else:
            # Handle authors with multiple articles by joining them with commas
            article_ids = fh_matches["article_id"].astype(str).tolist()
            article_id_str = ",".join(article_ids)
            first_authors_covar.loc[first_authors_covar["first_author_key"] == aut_key, "article_id"] = article_id_str
            first_authors_covar.loc[first_authors_covar["first_author_key"] == aut_key, "Articles"] = len(article_ids)

In [27]:
if do_alt:
    for aut_key in first_authors_covar["first_author_key"]:
        if len(all_LH[all_LH["first_author_key"] == aut_key]) != 1:
            print (f"outer: 💥 {aut_key}, {len(all_LH[all_LH['first_author_key'] == aut_key])}")
        if len(all_LH_inner[all_LH_inner["first_author_key"] == aut_key]) != 1:
            print (f"inner: 💥 {aut_key}, {len(all_LH_inner[all_LH_inner['first_author_key'] == aut_key])}")

## Summary
So how to analyze ? The key works well to merge at least, so let's have it. I think we should not do by authors because the aurhors are fleeting. Whether it's first or not. Let's do it by claim when it is needed. I think it's much more correct.

## Deprecated: author_pairs file!


### B.1. Read the Triage last_author files which contains all pairs

update April 7, 2025: we don't care avout this file anymore

In [30]:
# Read Author info, which contains all the pairs
paper_auth_pairs = pd.read_excel('input_data/2025-02-14_last_xlsx/1_Triage_Last author.xlsx', sheet_name='Tri sans les doublons')
# Drop all columns with 'Unnamed' in the name
paper_auth_pairs = paper_auth_pairs.drop(columns=[col for col in paper_auth_pairs.columns if 'Unnamed' in col]).drop(columns=['Source'])
# rename RIZKI MT	to Rizki TM in column last author
paper_auth_pairs['last author'] = paper_auth_pairs['last author'].replace({'Rizki MT': 'Rizki TM','RIZKI MT': 'Rizki TM' })

paper_auth_pairs.loc[paper_auth_pairs["MD"] == "??", "MD"] = np.nan
paper_auth_pairs.loc[paper_auth_pairs["MD"] == "?", "MD"] = np.nan
paper_auth_pairs.loc[paper_auth_pairs["MD"] == 'O', "MD"] = False
paper_auth_pairs.loc[paper_auth_pairs["MD"] == 0, "MD"] = False
paper_auth_pairs.loc[paper_auth_pairs["MD"] == 1, "MD"] = True

paper_auth_pairs.loc[paper_auth_pairs["Become a Pi"] == "??", "Become a Pi"] = np.nan
paper_auth_pairs.loc[paper_auth_pairs["Become a Pi"] == 0, "Become a Pi"] = False
paper_auth_pairs.loc[paper_auth_pairs["Become a Pi"] == 1, "Become a Pi"] = True

paper_auth_pairs.loc[paper_auth_pairs["Ivy league"] == 0, "Ivy league"] = False
paper_auth_pairs.loc[paper_auth_pairs["Ivy league"] == 1, "Ivy league"] = True

paper_auth_pairs.loc[paper_auth_pairs["Sex"] == "??", "Sex"] = np.nan
paper_auth_pairs.loc[paper_auth_pairs["Sex"] == 1, "Sex"] = "Male"
paper_auth_pairs.loc[paper_auth_pairs["Sex"] == 0, "Sex"] = "Female"

paper_auth_pairs.loc[paper_auth_pairs["current job"] == "pi", "current job"] = "PI"
paper_auth_pairs.loc[paper_auth_pairs["current job"] == "Pi", "current job"] = "PI"
paper_auth_pairs.loc[paper_auth_pairs["current job"] == "?", "current job"] = np.nan
paper_auth_pairs.loc[paper_auth_pairs["current job"] == "??", "current job"] = np.nan

# create merge columns: lowercased and stripped of accents
paper_auth_pairs['first_author_key'] = paper_auth_pairs['first author'].str.lower()
paper_auth_pairs['first_author_key'] = paper_auth_pairs['first_author_key'].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')
paper_auth_pairs = preprocess_utils.clean_author_keys(paper_auth_pairs, "first_author_key")

paper_auth_pairs.loc[:, 'last_author_key'] = paper_auth_pairs['last author'].str.lower()
paper_auth_pairs.loc[:, 'last_author_key'] = (paper_auth_pairs['last_author_key']
    .str.normalize('NFKD')
    .str.encode('ascii', errors='ignore')
    .str.decode('utf-8'))
#paper_auth_pairs.loc[:, 'last_author_key'] = paper_auth_pairs['last_author_key'].str.replace('rizki mt', 'rizki tm')
paper_auth_pairs = preprocess_utils.clean_author_keys(paper_auth_pairs, "last_author_key")


print(paper_auth_pairs["Sex"].unique(), paper_auth_pairs["MD"].unique(), paper_auth_pairs["Become a Pi"].unique(), paper_auth_pairs["Ivy league"].unique())

paper_auth_pairs = paper_auth_pairs.apply(preprocess_utils.safe_strip)


['Female' 'Male' nan] [False nan True] [True False nan] [True False]


/var/folders/pj/15hxgl0j1wg2w_t5k7bvnq5c0000gn/T/ipykernel_49277/2740770010.py:18: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'False' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  paper_auth_pairs.loc[paper_auth_pairs["Ivy league"] == 0, "Ivy league"] = False


In [31]:
paper_auth_pairs

,last author,first author,Sex,PhD Post-doc,Become a Pi,current job,MD,Affiliation,Country,Ivy league,first_author_key,last_author_key
0,Agaisse H,Derré,Female,Post-doc,True,PI,False,Yale University,USA,True,derre i,agaisse h
1,Aguilera RJ,Seong CS,Male,PhD,False,Academia,False,The University of Texas,USA,False,seong cs,aguilera rj
2,Aigaki T,Tsuda M,Male,PhD,True,Admin,False,Tokyo Metropolitan University,Japan,False,tsuda m,aigaki t
3,Ando,Markus,Male,PhD,False,Facility leader,False,"Hungarian Academy of Sciences, Szeged",Hungary,False,markus r,ando i
4,Ando,Rus F,Female,PhD,False,NaN,False,"Hungarian Academy of Sciences, Szeged",Hungary,False,rus f,ando i
...,...,...,...,...,...,...,...,...,...,...,...,...
315,Yu XQ,Ao J,NaN,PhD,NaN,Senior Staff,False,niversity of Missouri-Kansas City,USA,False,ao j,yu xq
316,Zhu S,Yuan Y,Male,PhD,False,Senior Staff,False,"Institute of Zoology, Chinese Academy of Sciences",China,False,yuan y,zhu s
317,Zhu S,Tian C,Female,PhD,True,PI,False,"Institute of Zoology, Chinese Academy of Sciences",China,False,tian c,zhu s
318,Zhu S,Zhang Z,Female,PhD,False,Academia,False,"Institute of Zoology, Chinese Academy of Sciences",China,False,zhang z,zhu s


In [ ]:
paper_auth_pairs

,last author,first author,Sex,PhD Post-doc,Become a Pi,current job,MD,Affiliation,Country,Ivy league,first_author_key,last_author_key
0,Agaisse H,Derré,Female,Post-doc,True,PI,False,Yale University,USA,True,derre i,agaisse h
1,Aguilera RJ,Seong CS,Male,PhD,False,Academia,False,The University of Texas,USA,False,seong cs,aguilera rj
2,Aigaki T,Tsuda M,Male,PhD,True,Admin,False,Tokyo Metropolitan University,Japan,False,tsuda m,aigaki t
3,Ando,Markus,Male,PhD,False,Facility leader,False,"Hungarian Academy of Sciences, Szeged",Hungary,False,markus r,ando i
4,Ando,Rus F,Female,PhD,False,NaN,False,"Hungarian Academy of Sciences, Szeged",Hungary,False,rus f,ando i
...,...,...,...,...,...,...,...,...,...,...,...,...
315,Yu XQ,Ao J,NaN,PhD,NaN,Senior Staff,False,niversity of Missouri-Kansas City,USA,False,ao j,yu xq
316,Zhu S,Yuan Y,Male,PhD,False,Senior Staff,False,"Institute of Zoology, Chinese Academy of Sciences",China,False,yuan y,zhu s
317,Zhu S,Tian C,Female,PhD,True,PI,False,"Institute of Zoology, Chinese Academy of Sciences",China,False,tian c,zhu s
318,Zhu S,Zhang Z,Female,PhD,False,Academia,False,"Institute of Zoology, Chinese Academy of Sciences",China,False,zhang z,zhu s


In [ ]:
print(len(authors.article_id.unique()))
authors

400


,author_id,article_id,name,sex,leading_author,first_author,career_stage,expertise_level,author_key
0,2,2053,Brun S,Male,False,True,Post-doc,Experienced,brun s
1,3,2053,Lemaitre B,Male,True,False,Junior PI,Experienced,lemaitre b
2,1,1650,Charroux B,Male,False,True,Post-doc,Experienced,charroux b
3,97,2149,Söderhäll K,Male,True,False,Senior PI,Experienced,soderhall k
4,5,2278,Silverman N,Male,False,True,Post-doc,Newcomer,silverman n
...,...,...,...,...,...,...,...,...,...
792,797,2067,Cerenius L,Male,True,False,Senior PI,Experienced,cerenius l
793,798,2122,Higgins DE,Male,True,False,Senior PI,Newcomer,higgins de
794,799,2184,Mengin-Lecreulx D,Male,True,False,Senior PI,Experienced,mengin-lecreulx d
795,800,2130,Moore KJ,Female,True,False,Junior PI,Newcomer,moore kj


In [ ]:
paper_auth_pairs["article_id"] = pd.NA
paper_auth_pairs["article_id"] = paper_auth_pairs["article_id"].astype("Int64")

In [ ]:
for art in authors["article_id"].unique():
    these_authors = authors[authors["article_id"] == art]
    these_first_authors = these_authors[these_authors["first_author"] == True]["author_key"].values
    these_leading_authors = these_authors[these_authors["leading_author"] == True]["author_key"].values
    if (len(these_first_authors) == 1 and len(these_leading_authors) == 1):
        indexes = paper_auth_pairs[(paper_auth_pairs["last_author_key"] == these_leading_authors[0]) & 
                                (paper_auth_pairs["first_author_key"] == these_first_authors[0])].index.to_list()
        if len(indexes) == 1:
            paper_auth_pairs.loc[indexes[0], "article_id"] = art
        elif len(indexes) > 1:
            first_na = pd.isna(paper_auth_pairs.loc[indexes, "article_id"]).idxmax() #idmax returns the index of the first occurrence of the maximum value
            print(these_authors["author_key"].values, pd.isna(paper_auth_pairs.loc[indexes, "article_id"]).values, first_na)
            paper_auth_pairs.loc[first_na, "article_id"] = art
        else:
            print(f"NO MATCH 🛑 {art}, {len(indexes)}, {these_first_authors[0]:<20} -- {these_leading_authors[0]:<20} ... {len(these_authors)}")
    else:
        print(f"SKIPPING {art} because only one={len(these_authors)} author: {these_authors['author_key'].values}")

['georgel p' 'hoffmann ja'] [ True  True] 96
NO MATCH 🛑 2239, 0, hedengren-olcott m   -- taylor bj            ... 2
['georgel p' 'hoffmann ja'] [False  True] 101
NO MATCH 🛑 1947, 0, schneider ds         -- schneider ds         ... 1
NO MATCH 🛑 2730, 0, schneider ds         -- schneider ds         ... 2
NO MATCH 🛑 3447, 0, rizki tm             -- rizki tm             ... 1
NO MATCH 🛑 3448, 0, rizki tm             -- rizki tm             ... 1
NO MATCH 🛑 3292, 0, rizki tm             -- rizki tm             ... 1
NO MATCH 🛑 3273, 0, rizki tm             -- rizki tm             ... 2
['georgel p' 'hoffmann ja'] [False False] 96
NO MATCH 🛑 2764, 0, imler jl             -- imler jl             ... 1
NO MATCH 🛑 3446, 0, nappi aj             -- nappi aj             ... 1


### There is an error above, : 3 hoffman georgel, whereas in the excel there are two...

In [ ]:
paper_auth_pairs[paper_auth_pairs["article_id"].isna()]

,last author,first author,Sex,PhD Post-doc,Become a Pi,current job,MD,Affiliation,Country,Ivy league,first_author_key,last_author_key,article_id
16,Bellotti RA,Rizki TM,Male,PI,True,Deceased,False,Ann arbor,USA,False,rizki tm,bellotti ra,<NA>
106,Hoffmann JA,Imler JL,Male,PI,True,PI,False,CNRS-University of Strasbourg,France,False,imler jl,hoffmann ja,<NA>
240,Rizki TM,RIZKI MT,Male,PI,True,Retired,False,ann arbor,NaN,False,rizki mt,rizki tm,<NA>
271,Shahabuddin M,Schneider D,Male,Pi,True,PI,True,Whitehead Institute,USA,False,schneider d,shahabuddin m,<NA>
272,Shirasu-Hiza MM,Schneider DS,Male,Pi,True,PI,True,Stanford University Stanford,USA,True,schneider ds,shirasu-hiza mm,<NA>
278,Silvers MJ,Nappi AJ,Male,PI,True,PI,False,Univeristy of Chicago,USA,False,nappi aj,silvers mj,<NA>
290,Taylor BJ,Hedengren M,Female,PhD,False,NaN,False,Oregon State University,USA,False,hedengren m,taylor bj,<NA>


In [ ]:
paper_auth_pairs.loc[290, "article_id"] = 2239

In [ ]:
paper_auth_pairs[paper_auth_pairs["article_id"].isna()]

,last author,first author,Sex,PhD Post-doc,Become a Pi,current job,MD,Affiliation,Country,Ivy league,first_author_key,last_author_key,article_id
16,Bellotti RA,Rizki TM,Male,PI,True,Deceased,False,Ann arbor,USA,False,rizki tm,bellotti ra,<NA>
106,Hoffmann JA,Imler JL,Male,PI,True,PI,False,CNRS-University of Strasbourg,France,False,imler jl,hoffmann ja,<NA>
240,Rizki TM,RIZKI MT,Male,PI,True,Retired,False,ann arbor,NaN,False,rizki mt,rizki tm,<NA>
271,Shahabuddin M,Schneider D,Male,Pi,True,PI,True,Whitehead Institute,USA,False,schneider d,shahabuddin m,<NA>
272,Shirasu-Hiza MM,Schneider DS,Male,Pi,True,PI,True,Stanford University Stanford,USA,True,schneider ds,shirasu-hiza mm,<NA>
278,Silvers MJ,Nappi AJ,Male,PI,True,PI,False,Univeristy of Chicago,USA,False,nappi aj,silvers mj,<NA>


In [ ]:
len(paper_auth_pairs.article_id.unique())

315

In [ ]:
first_authors_claims

,Name,Sex,Articles,Major claims,Unchallenged,Verified,Partially Verified,Mixed,Challenged,Authorship,first_author_key
0,Williams MJ,Male,5,14,7,3,1,0,3,First,williams mj
1,Lemaitre B,Male,4,12,0,11,1,0,0,First,lemaitre b
2,Ha EM,Female,4,10,3,1,2,2,2,First,ha em
3,Leulier F,Male,4,10,0,10,0,0,0,First,leulier f
4,De Gregorio E,Male,3,9,1,8,0,0,0,First,de gregorio e
...,...,...,...,...,...,...,...,...,...,...,...
287,Schlenke TA,Male,1,1,0,1,0,0,0,First,schlenke ta
288,Uttenweiler-Joseph S,Female,1,1,0,1,0,0,0,First,uttenweiler-joseph s
289,Wang LN,Female,1,1,0,1,0,0,0,First,wang ln
290,Yuan Y,Male,1,1,0,1,0,0,0,First,yuan y


### B.3 Merge to create a covariate file for Last Author

In [ ]:
paper_auth_pairs_LH = paper_auth_pairs[["last author", "last_author_key", "Affiliation", "Country", "Ivy league"]]
paper_auth_pairs_LH = preprocess_utils.deduplicate_by(paper_auth_pairs_LH, "last author").copy()

In [ ]:
 # check for duplicate last_author_key in paper_auth_pairs_LH
paper_auth_pairs_LH[paper_auth_pairs_LH.duplicated('last_author_key', keep=False)]
print(len(paper_auth_pairs_LH[paper_auth_pairs_LH.duplicated('last_author_key', keep=False)]), "duplicates in paper_auth_pairs_LH")
# check for duplicate last_author_key in leading_authors_claims
print(len(leading_authors_claims[leading_authors_claims.duplicated('last_author_key', keep=False)]), "duplicates in leading_authors_claims")

0 duplicates in paper_auth_pairs_LH
0 duplicates in leading_authors_claims


In [ ]:
# Perform the outer merge to see what we are missing
all_LH = pd.merge(leading_authors_claims, paper_auth_pairs_LH, on='last_author_key', how='outer')
print(len(leading_authors_claims), len(paper_auth_pairs_LH), len(all_LH))

157 160 161


In [ ]:
unique_pairs = all_LH[["Name", "last author", "last_author_key"]].drop_duplicates().sort_values("last_author_key", ascending=True)
print(" ONLY in Pairs               only in Claims        matching key ")
print("-"*70)
for i in range(0, len(unique_pairs)):
    if pd.isna(unique_pairs.iloc[i]['last author']) or pd.isna(unique_pairs.iloc[i]['Name']):
        print('💥 ', end='')
        print(f"{unique_pairs.iloc[i]['last author']:<20} vs  {unique_pairs.iloc[i]['Name']:<20} --> {unique_pairs.iloc[i]['last_author_key']:<20}")
    #else:
    #    print(f"   {unique_pairs.iloc[i]['last author']:<20} vs  {unique_pairs.iloc[i]['Name']:<20} --> {unique_pairs.iloc[i]['last_author_key']:<20}")

 ONLY in Pairs               only in Claims        matching key 
----------------------------------------------------------------------
💥 Bellotti RA          vs  nan                  --> bellotti ra         
💥 nan                  vs  Nappi AJ             --> nappi aj            
💥 Shahabuddin M        vs  nan                  --> shahabuddin m       
💥 Shirasu-Hiza MM      vs  nan                  --> shirasu-hiza mm     
💥 Silvers MJ           vs  nan                  --> silvers mj          


In [ ]:
all_LH_inner = pd.merge(leading_authors_claims, paper_auth_pairs_LH, on='last_author_key', how='inner').sort_values("last_author_key", ascending=True)
#all_LH_inner.drop(columns=['last_author_key', 'last author', 'Authorship'], inplace=True)
print(len(all_LH_inner))


assessment_columns = ['Unchallenged', 'Verified', 'Partially Verified', 'Mixed', 'Challenged']
# set type of assessment columns to int
for col in assessment_columns:
    all_LH_inner[col] = all_LH_inner[col].astype(int)
all_LH_inner['Major claims'] = all_LH_inner['Major claims'].astype(int)
all_LH_inner['Articles'] = all_LH_inner['Articles'].astype(int)

for col in assessment_columns:
    all_LH_inner[f'{col}_prop'] = all_LH_inner[col] / all_LH_inner['Major claims']

all_LH_inner = all_LH_inner.apply(preprocess_utils.safe_strip)
all_LH_inner.to_csv('preprocessed_data/LH_inner.csv', index=False)
all_LH_inner

156


,Name,Historical lab,Historical lab after 1998,Continuity,F and L,Sex,Articles,Major claims,Unchallenged,Verified,...,last_author_key,last author,Affiliation,Country,Ivy league,Unchallenged_prop,Verified_prop,Partially Verified_prop,Mixed_prop,Challenged_prop
63,Agaisse H,False,0.0,False,1.0,Male,1,3,2,1,...,agaisse h,Agaisse H,Yale University,USA,True,0.666667,0.333333,0.000000,0.0,0.0
107,Aguilera RJ,False,0.0,False,0.0,Male,1,2,0,1,...,aguilera rj,Aguilera RJ,The University of Texas,USA,False,0.000000,0.500000,0.000000,0.0,0.5
64,Aigaki T,False,0.0,False,0.0,Male,1,3,2,1,...,aigaki t,Aigaki T,Tokyo Metropolitan University,Japan,False,0.666667,0.333333,0.000000,0.0,0.0
6,Anderson KV,False,0.0,False,0.0,Female,8,20,7,11,...,anderson kv,Anderson KV,Memorial Sloan-Kettering Cancer Center and the...,USA,True,0.350000,0.550000,0.000000,0.0,0.1
16,Andó I,False,0.0,True,0.0,Male,6,12,1,7,...,ando i,Ando,"Hungarian Academy of Sciences, Szeged",Hungary,False,0.083333,0.583333,0.333333,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
104,Xu T,False,0.0,False,0.0,Male,1,3,0,3,...,xu t,Xu T,Yale University School of Medicine,USA,False,0.000000,1.000000,0.000000,0.0,0.0
146,Yamaguchi M,False,0.0,False,0.0,Male,1,2,1,1,...,yamaguchi m,Yamaguchi M,Aichi Cancer Center Research Institute,Japan,False,0.500000,0.500000,0.000000,0.0,0.0
105,Yoo MA,False,0.0,False,0.0,Female,1,3,0,2,...,yoo ma,Yoo MA,Pusan National University,Korea,False,0.000000,0.666667,0.333333,0.0,0.0
106,Yu XQ,False,0.0,False,0.0,Male,1,3,1,2,...,yu xq,Yu XQ,niversity of Missouri-Kansas City,USA,False,0.333333,0.666667,0.000000,0.0,0.0


In [ ]:
for aut in authors[authors["leading_author"]]["author_key"]:
    if len(all_LH[all_LH["last_author_key"] == aut]) != 1:
        print (f"outer: 💥 {aut}, {len(all_LH[all_LH['last_author_key'] == aut])}")
    if len(all_LH_inner[all_LH_inner["last_author_key"] == aut]) != 1:
        print (f"inner: 💥 {aut}, {len(all_LH_inner[all_LH_inner['last_author_key'] == aut])}")


inner: 💥 nappi aj, 0


### B. First authors - deprecated

In [ ]:
all_FH = pd.merge(first_authors_claims, first_authors_claims2, on='first_author_key', how='outer', suffixes=('_claims', '_claims2'))
all_LH_inner = pd.merge(first_authors_claims, first_authors_claims2, on='first_author_key', how='inner').sort_values("first_author_key", ascending=True)
print(len(first_authors_claims), len(first_authors_claims2), len(all_FH), len(all_LH_inner))

292 290 295 291


In [ ]:
first_authors_claims2.Affiliation.value_counts()

Affiliation
CNRS-University of Strasbourg                  32
University of Stockholm                        29
CNRS_Gif-sur-Yvette_Université Paris-Saclay    20
University of Massachusetts Medical School     15
CNRS University of Marseille                    7
                                               ..
University of Uppsala                           1
Kansas State University                         1
University of Modena and Reggio Emilia          1
University of Glasgow                           1
Shanghai Jiao Tong University                   1
Name: count, Length: 109, dtype: int64

In [ ]:
paper_auth_pairs_FH = paper_auth_pairs.drop(columns=['last author']).sort_values("first author", ascending=True)
paper_auth_pairs_FH

,first author,Sex,PhD Post-doc,Become a Pi,current job,MD,Affiliation,Country,Ivy league,first_author_key,last_author_key
243,Abdelsadik A,Male,Senior Staff,True,PI,False,Forschungszentrum Borstel,Germany,False,abdelsadik a,roeder t
220,Agaisse H,Male,Post-doc,True,PI,False,Harvard Medical School,USA,True,agaisse h,perrimon n
92,Agaisse H,Male,Post-doc,True,PI,False,Harvard Medical School,USA,True,agaisse h,higgins de
274,Aggarwal K,Female,PhD,False,Industry,False,University of Massachusetts Medical School,USA,False,aggarwal k,silverman n
315,Ao J,NaN,PhD,NaN,Senior Staff,False,niversity of Missouri-Kansas City,USA,False,ao j,yu xq
...,...,...,...,...,...,...,...,...,...,...,...
218,Zhang Z,Female,PhD,False,Admin,False,University of Kentucky,USA,False,zhang z,palli sr
87,Zhao HW,Female,PhD,False,NaN,False,University of California San Diego,USA,True,zhao hw,haddad gg
141,Zhou Z,Male,PhD,NaN,NaN,False,University of Houston,USA,False,zhou z,kimbrell da
79,Zhuang ZH,Male,PhD,False,NaN,False,Shanghai Jiao Tong University,China,False,zhuang zh,ge bx


In [ ]:
paper_auth_pairs_FH = preprocess_utils.deduplicate_by(paper_auth_pairs_FH, "first author")

            among ['perrimon n', 'higgins de'].
            among ['Post-doc', 'PhD'].
            among ['Harvard Medical School and Massachusetts General Hospital', 'Massachusetts General Hospital'].
            among ['rahme lg', 'mylonakis e'].
            among ['PhD', 'Post-doc'].
            among ['CEA Grenoble', 'Université de Toulouse'].
            among ['fauvarque mo', 'waltzer l'].
            among ['CNRS Gif-sur-Yvette Université Paris-Saclay', 'CNRS_Gif-sur-Yvette_Université Paris-Saclay'].
            among ['boccard f', 'lemaitre b'].
            among ['CNRS_Gif-sur-Yvette_Université Paris-Saclay', 'CNRS Gif-sur-Yvette Université Paris-Saclay'].
            among ['lemaitre b', 'brey pt'].
            among ['lagueux m', 'hoffmann ja'].
            among ['Massachusetts General Hospital', 'Harvard Medical School,Boston'].
            among ['white k', 'ezekowitz ra'].
            among ['Univeristy of Freiburg', 'Johannes Gutenberg Universität'].
            among [

In [ ]:

# check for duplicate first_author_key in paper_auth_pairs_LH
print(len(paper_auth_pairs_FH[paper_auth_pairs_FH.duplicated('first_author_key', keep=False)]), "duplicates in paper_auth_pairs_LH")
# check for duplicate first_author_key in leading_authors_claims
print(len(first_authors_claims[first_authors_claims.duplicated('first_author_key', keep=False)]), "duplicates in leading_authors_claims")


all_FH = pd.merge(first_authors_claims, paper_auth_pairs_FH, on='first_author_key', how='outer', suffixes=('', '_pairs'))
print(len(first_authors_claims), len(paper_auth_pairs_FH), len(all_FH))

print(" ONLY in Pairs               only in Claims        matching key ")
print("-"*70)
unique_pairs = all_FH[["Name", "first author", "first_author_key"]].drop_duplicates().sort_values("first_author_key", ascending=True)
for i in range(0, len(unique_pairs)):
    if pd.isna(unique_pairs.iloc[i]['first author']) or pd.isna(unique_pairs.iloc[i]['Name']):
        print('💥 ', end='')
        print(f" {unique_pairs.iloc[i]['first author']:<20}  {unique_pairs.iloc[i]['Name']:<20} --> {unique_pairs.iloc[i]['first_author_key']:<20}")

0 duplicates in paper_auth_pairs_LH
4 duplicates in leading_authors_claims
292 291 294
 ONLY in Pairs               only in Claims        matching key 
----------------------------------------------------------------------
💥  nan                   Hedengren-Olcott M   --> hedengren-olcott m  
💥  RIZKI MT              nan                  --> rizki mt            
💥  Schneider D           nan                  --> schneider d         


In [ ]:


all_FH = all_FH.apply(preprocess_utils.safe_strip)

In [ ]:
all_FH_inner = pd.merge(first_authors_claims, paper_auth_pairs_FH, on='first_author_key', how='inner', suffixes=('', '_pairs')).sort_values("first_author_key", ascending=True)
for idx, row in all_FH_inner.iterrows():
    if row["Sex"] != row["Sex_pairs"]:
        print(f"confligcthing sex for {row['Name']} : {row['Sex']} vs  {row['Sex_pairs']}")
all_FH_inner.drop(columns=["Sex_pairs"], inplace=True)
print(len(all_FH_inner))
all_FH_inner = all_FH_inner.apply(preprocess_utils.safe_strip)

assessment_columns = ['Unchallenged', 'Verified', 'Partially Verified', 'Mixed', 'Challenged']
# set type of assessment columns to int
for col in assessment_columns:
    all_FH_inner[col] = all_FH_inner[col].astype(int)
all_FH_inner['Major claims'] = all_FH_inner['Major claims'].astype(int)
all_FH_inner['Articles'] = all_FH_inner['Articles'].astype(int)

for col in assessment_columns:
    all_FH_inner[f'{col}_prop'] = all_FH_inner[col] / all_FH_inner['Major claims']

all_FH_inner.to_csv('preprocessed_data/FH_inner.csv', index=False)
all_FH_inner

confligcthing sex for Ao J : Male vs  nan
confligcthing sex for Avila A : Male vs  Female
confligcthing sex for Cheng W : Female vs  nan
confligcthing sex for Chiu H : Male vs  Female
confligcthing sex for Daffre S : Male vs  Female
confligcthing sex for Ertürk-Hasdemir D : Male vs  Female
confligcthing sex for Han SH : Male vs  Female
confligcthing sex for Hedengren M : Male vs  Female
confligcthing sex for Kim YS : Female vs  Male
confligcthing sex for Kwon EJ : Male vs  Female
confligcthing sex for Lindmark H : Female vs  Male
confligcthing sex for Luce-Fedrow A : Female vs  Male
confligcthing sex for Michel T : Female vs  Male
confligcthing sex for Mishima Y : Female vs  Male
confligcthing sex for Nam HJ : Male vs  Female
confligcthing sex for Peng J : Male vs  Female
confligcthing sex for Persson C : Male vs  Female
confligcthing sex for Pham LN : Male vs  Female
confligcthing sex for Ren C : Female vs  Male
confligcthing sex for Schmidt RL : Female vs  Male
confligcthing sex for 

,Name,Sex,Articles,Major claims,Unchallenged,Verified,Partially Verified,Mixed,Challenged,Authorship,...,MD,Affiliation,Country,Ivy league,last_author_key,Unchallenged_prop,Verified_prop,Partially Verified_prop,Mixed_prop,Challenged_prop
79,Abdelsadik A,Male,1,3,3,0,0,0,0,First,...,False,Forschungszentrum Borstel,Germany,False,roeder t,1.000000,0.000000,0.000000,0.0,0.0
19,Agaisse H,Male,2,6,2,4,0,0,0,First,...,False,Harvard Medical School,USA,True,perrimon n,0.333333,0.666667,0.000000,0.0,0.0
195,Aggarwal K,Female,1,2,0,2,0,0,0,First,...,False,University of Massachusetts Medical School,USA,False,silverman n,0.000000,1.000000,0.000000,0.0,0.0
80,Ao J,Male,1,3,1,2,0,0,0,First,...,False,niversity of Missouri-Kansas City,USA,False,yu xq,0.333333,0.666667,0.000000,0.0,0.0
20,Apidianakis Y,Male,3,6,4,2,0,0,0,First,...,False,Harvard Medical School and Massachusetts Gener...,USA,True,rahme lg,0.666667,0.333333,0.000000,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
193,Zettervall CJ,Male,1,3,0,3,0,0,0,First,...,False,University of Stockholm,Sweden,False,hultmark d,0.000000,1.000000,0.000000,0.0,0.0
273,Zhang Z,Female,2,2,1,1,0,0,0,First,...,False,"Institute of Zoology, Chinese Academy of Sciences",China,False,zhu s,0.500000,0.500000,0.000000,0.0,0.0
194,Zhao HW,Female,1,3,2,1,0,0,0,First,...,False,University of California San Diego,USA,True,haddad gg,0.666667,0.333333,0.000000,0.0,0.0
290,Zhou Z,Male,1,1,0,1,0,0,0,First,...,False,University of Houston,USA,False,kimbrell da,0.000000,1.000000,0.000000,0.0,0.0


In [ ]:
for aut in authors[authors["first_author"]]["author_key"]:
    #if len(all_FH[all_LH["first_author_key"] == aut]) != 1:
    #    print (f"outer: 💥 {aut}, {len(all_FH[all_FH['author_key'] == aut])}")
    if len(all_FH_inner[all_FH_inner["first_author_key"] == aut]) != 1:
        print (f"inner: 💥 {aut}, {len(all_FH_inner[all_FH_inner['first_author_key'] == aut])}")

inner: 💥 hedengren-olcott m, 0
inner: 💥 hedengren m, 2
inner: 💥 hedengren m, 2
inner: 💥 kim ys, 2
inner: 💥 kim ys, 2
inner: 💥 kim ys, 2


In [ ]:
all_FH_inner.columns

Index(['Name', 'Sex', 'Articles', 'Major claims', 'Unchallenged', 'Verified',
       'Partially Verified', 'Mixed', 'Challenged', 'Authorship',
       'first_author_key', 'first author', 'PhD Post-doc', 'Become a Pi',
       'current job', 'MD', 'Affiliation', 'Country', 'Ivy league',
       'last_author_key', 'Unchallenged_prop', 'Verified_prop',
       'Partially Verified_prop', 'Mixed_prop', 'Challenged_prop'],
      dtype='object')

In [ ]:
for col in all_FH_inner.columns:
    print(70*"-")
    print(all_FH_inner[col].value_counts())